## 1. Python librairies

In [3]:
import io
import json
import os
import requests
import gradio as gr

from dotenv import load_dotenv
from PIL import Image

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import Vector

from azure.storage.blob import BlobServiceClient

import warnings
from tqdm import TqdmWarning

warnings.filterwarnings('ignore', category=TqdmWarning)

## 2. Azure AI Services

In [5]:
load_dotenv("azure.env")

acv_key = os.getenv("acv_key")
acv_endpoint = os.getenv("acv_endpoint")

acs_endpoint = os.getenv("acs_endpoint")
acs_key = os.getenv("acs_key")

if acv_endpoint.endswith("/"):
    acv_endpoint = acv_endpoint[:-1]

if acs_endpoint.endswith("/"):
    acs_endpoint = acv_endpoint[:-1]
    
blob_connection_string = os.getenv("blob_connection_string")
container_name = os.getenv("container_name")

index_name = "fashion-demo"

api_version = "2023-02-01-preview"

## 3. Functions & parameters

In [6]:
def text_embedding(prompt):
    version = "?api-version=" + api_version + "&modelVersion=latest"
    vec_txt_url = f"{acv_endpoint}/computervision/retrieval:vectorizeText{version}"
    headers = {"Content-type": "application/json", "Ocp-Apim-Subscription-Key": acv_key}

    payload = {"text": prompt}
    response = requests.post(vec_txt_url, json=payload, headers=headers)

    if response.status_code == 200:
        text_emb = response.json().get("vector")
        return text_emb

    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [7]:
def index_stats(index_name):
    url = (
        acs_endpoint
        + "/indexes/"
        + index_name
        + "/stats?api-version=2021-04-30-Preview"
    )
    headers = {
        "Content-Type": "application/json",
        "api-key": acs_key,
    }
    response = requests.get(url, headers=headers)
    print("Azure Cognitive Search index status for:", index_name, "\n")

    if response.status_code == 200:
        res = response.json()
        print(json.dumps(res, indent=2))

    else:
        print("Request failed with status code:", response.status_code)

In [8]:
def index_status(index_name):
    print("Azure Cognitive Search Index:", index_name, "\n")

    headers = {"Content-Type": "application/json", "api-key": acs_key}
    params = {"api-version": "2021-04-30-Preview"}
    index_status = requests.get(
        acs_endpoint + "/indexes/" + index_name, headers=headers, params=params
    )
    try:
        print(json.dumps((index_status.json()), indent=5))
    except:
        print("Request failed")

In [10]:
blob_service_client = BlobServiceClient.from_connection_string(blob_connection_string)
container_client = blob_service_client.get_container_client(container_name)
blobs = container_client.list_blobs()

## 4. Azure Cognitive Search

In [11]:
try:
    print("Setting the Azure Cognitive Search client")
    search_client = SearchIndexClient(
        endpoint=acs_endpoint, credential=AzureKeyCredential(acs_key)
    )
    print("Done")
    print(search_client)

except:
    print("Request failed. Cannot create Azure Cognitive Search client:", acs_endpoint)

Setting the Azure Cognitive Search client
Done


### Azure Cognitive Search index status

In [12]:
index_stats(index_name)

Azure Cognitive Search index status for: fashion-demo 

{
  "@odata.context": "https://mono-cognitive-search.search.windows.net/$metadata#Microsoft.Azure.Search.V2021_04_30_Preview.IndexStatistics",
  "documentCount": 25879,
  "storageSize": 394790772
}


In [13]:
index_status(index_name)

Azure Cognitive Search Index: fashion-demo 

{
     "@odata.context": "https://mono-cognitive-search.search.windows.net/$metadata#indexes/$entity",
     "@odata.etag": "\"0x8DC02877B219591\"",
     "name": "fashion-demo",
     "defaultScoringProfile": null,
     "fields": [
          {
               "name": "idfile",
               "type": "Edm.String",
               "searchable": false,
               "filterable": false,
               "retrievable": true,
               "sortable": false,
               "facetable": false,
               "key": true,
               "indexAnalyzer": null,
               "searchAnalyzer": null,
               "analyzer": null,
               "normalizer": null,
               "synonymMaps": []
          },
          {
               "name": "imagefile",
               "type": "Edm.String",
               "searchable": true,
               "filterable": false,
               "retrievable": true,
               "sortable": false,
               "facet

## 5. Webapp

In [14]:
import base64
import os

def get_image_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

topn = 20
imgsize = 360

footnote = "Powered by Tarık Kaan koç"

logo_path = os.path.join(os.getcwd(), "assets", "workflow.png")
logo_base64 = get_image_base64(logo_path)
image = f'<center><img src="data:image/png;base64,{logo_base64}" width="512px"></center>'

theme = "snehilsanyal/scikit-learn"

In [15]:
import cv2

def image_embedding(imagefile):
    session = requests.Session()

    version = "?api-version=" + api_version + "&modelVersion=latest"
    vec_img_url = acv_endpoint + "/computervision/retrieval:vectorizeImage" + version
    headers = {
        "Content-type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": acv_key,
    }

    try:
        is_success, buffer = cv2.imencode(".jpg", imagefile)
        io_buf = io.BytesIO(buffer)
        data = io_buf.read()
        response = session.post(vec_img_url, data=data, headers=headers)
        response.raise_for_status()

        image_emb = response.json()["vector"]
        return image_emb

    except requests.exceptions.RequestException as e:
        print(f"Request Exception: {e}")
    except Exception as ex:
        print(f"Error: {ex}")

    return None


def image_search_gradio(imagefile, topn=20):
    results_list = []
    images_list = []

    search_client = SearchClient(acs_endpoint, index_name, AzureKeyCredential(acs_key))

    response = search_client.search(
        search_text="",
        vector=Vector(value=image_embedding(imagefile), k=topn, fields="imagevector"),
        select=["idfile", "imagefile"],
    )

    for result in response:
        image_file = result["imagefile"]
        results_list.append(image_file)

    for image_file in results_list:
        blob_client = container_client.get_blob_client(image_file)
        blob_image = blob_client.download_blob().readall()
        img = Image.open(io.BytesIO(blob_image)).resize((imgsize, imgsize))
        images_list.append(img)

    return images_list

def image_search_gradio(imagefile, topn=20):
    results_list = []
    images_list = []

    search_client = SearchClient(acs_endpoint, index_name, AzureKeyCredential(acs_key))

    response = search_client.search(
        search_text="",
        vector=Vector(value=image_embedding(imagefile), k=topn, fields="imagevector"),
        select=["idfile", "imagefile"],
    )

    for result in response:
        image_file = result["imagefile"]
        results_list.append(image_file)

    for image_file in results_list:
        blob_client = container_client.get_blob_client(image_file)
        blob_image = blob_client.download_blob().readall()
        img = Image.open(io.BytesIO(blob_image)).resize((imgsize, imgsize))
        images_list.append(img)

    return images_list


topn_list_prompt = [""] * topn

list_img_results_image = [
    gr.components.Image(label=f"Top {i+1} : {topn_list_prompt[i]}", type="filepath")
    for i in range(topn)
]

webapp_image = gr.Interface(
    image_search_gradio,
    gr.components.Image(),  
    list_img_results_image,
    theme=theme,
    description=image,
    article=footnote,
)

### Combining the two gradio apps into one

In [16]:
visual_search_webapp = gr.TabbedInterface(
    [webapp_image],
    [
        "Bir resim ile ürün ara"
    ],
    css="body {background-color: black}",
    theme=theme,
)

In [17]:
visual_search_webapp.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://32c7b37452499d001f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/Users/koc/Desktop/visual-search-azureAI/venv/lib/python3.9/site-packages/gradio/components/button.py:89: UserWarning: Using the update method is deprecated. Simply return a new object instead, e.g. `return gr.Button(...)` instead of `return gr.Button.update(...)`.
  warnings.warn(
